In [87]:
# python env: modnenv_v2

import os
import string
import random
import numpy as np
from sklearn.model_selection import KFold
from pathlib import Path, PosixPath
import matplotlib.pyplot as plt
import json 
import pandas as pd
from copy import deepcopy
from modnet.models import EnsembleMODNetModel
from modnet.preprocessing import MODData
from modnet.hyper_opt import FitGenetic
from monty.serialization import dumpfn, loadfn
from optimade.adapters import Structure as optim_Structure
from pymatgen.ext.matproj import MPRester
from pymatgen.core.structure import Structure
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error 
from scipy.stats import spearmanr
from IPython.display import Image
from tqdm import tqdm
from os import listdir
from os.path import isfile, join
import pickle
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [37]:
# path_alexandria = "/home/vtrinquet/Documents/Doctorat/JNB_Scripts_Clusters/NLO/HT/ref_idx/re2fractive/humanguided/alexandria"
# path_alexandria_strucs = (Path(path_alexandria) / "alexandria_pbesol_structures_start.json")
# with open(path_alexandria_strucs, "r") as f:
#     strucs_alex_start = json.load(f)

In [38]:
# path_alexandria = "/home/vtrinquet/Documents/Doctorat/JNB_Scripts_Clusters/NLO/HT/ref_idx/re2fractive/humanguided/alexandria"
# path_alexandria_strucs = (Path(path_alexandria) / "alexandria_pbesol_structures_end.json")
# with open(path_alexandria_strucs, "r") as f:
#     strucs_alex_end = json.load(f)

In [24]:
# strucs_alex_start['structures']['_alexandria_band_gap >= 0.3 AND _alexandria_hull_distance <= 0.025']['https://alexandria.icams.rub.de/pbesol'].keys()

dict_keys(['data', 'errors', 'links', 'included', 'meta'])

In [26]:
# len(strucs_alex_start['structures']['_alexandria_band_gap >= 0.3 AND _alexandria_hull_distance <= 0.025']['https://alexandria.icams.rub.de/pbesol']['data'])

20

In [27]:
# strucs_alex_end['structures']['_alexandria_band_gap >= 0.3 AND _alexandria_hull_distance <= 0.025']['https://alexandria.icams.rub.de/pbesol'].keys()

dict_keys(['data', 'errors', 'links', 'included', 'meta'])

In [28]:
# len(strucs_alex_end['structures']['_alexandria_band_gap >= 0.3 AND _alexandria_hull_distance <= 0.025']['https://alexandria.icams.rub.de/pbesol']['data'])

25672

In [30]:
# strucs_alex_start_data = strucs_alex_start['structures']['_alexandria_band_gap >= 0.3 AND _alexandria_hull_distance <= 0.025']['https://alexandria.icams.rub.de/pbesol']['data']
# strucs_alex_end_data = strucs_alex_end['structures']['_alexandria_band_gap >= 0.3 AND _alexandria_hull_distance <= 0.025']['https://alexandria.icams.rub.de/pbesol']['data']

In [33]:
# ids_start = []
# for inst in strucs_alex_start_data:
#     ids_start.append(inst['id'])

In [36]:
# count=0
# for inst in strucs_alex_end_data:
#     if inst['id'] in ids_start:
#         print(inst)
#         print(count)
#         count+=1

{'id': 'agm003211273', 'type': 'structures', 'attributes': {'immutable_id': 'agm003211273', 'last_modified': '2023-11-16T06:57:59Z', 'elements': ['F', 'Li', 'Lu'], 'nelements': 3, 'elements_ratios': [0.6666666666666666, 0.16666666666666666, 0.16666666666666666], 'chemical_formula_descriptive': 'Li2 Lu2 F8', 'chemical_formula_reduced': 'F4LiLu', 'chemical_formula_hill': None, 'chemical_formula_anonymous': 'A4BC', 'dimension_types': [1, 1, 1], 'nperiodic_dimensions': 3, 'lattice_vectors': [[5.07305744, 0.10882747, -0.0], [-0.12651635, 5.08041283, -0.0], [-2.47327054, -2.59462015, 5.16271436]], 'cartesian_site_positions': [[1.268264361239788, 0.027206867500000093, 1.2801339524282744], [1.2050061887602124, 2.5674132825000004, 3.882580407571725], [-1.2682643315190265, -0.027206841553798225, 3.870247457372839], [3.7415348815190277, 2.621826991553799, 1.2924669026271602], [2.6291446499767686, 3.454424919328984, 3.031679018595344], [0.3694343411881434, 1.456693111008315, 0.44818313254457076], 

In conclusion, all the 20 materials of the first part of the original json file are included in the second part so let's only consider the 2nd part of the json file, i.e., the bisbis one.

In [39]:
path_alexandria = "/home/vtrinquet/Documents/Doctorat/JNB_Scripts_Clusters/NLO/HT/ref_idx/re2fractive/humanguided/alexandria"
path_alexandria_strucs = (Path(path_alexandria) / "alexandria_pbesol_structures_end.json")
with open(path_alexandria_strucs, "r") as f:
    strucs_alex_end = json.load(f)

In [40]:
strucs_alex_end['structures']['_alexandria_band_gap >= 0.3 AND _alexandria_hull_distance <= 0.025']['https://alexandria.icams.rub.de/pbesol'].keys()

dict_keys(['data', 'errors', 'links', 'included', 'meta'])

In [42]:
strucs_alex_end_data = strucs_alex_end['structures']['_alexandria_band_gap >= 0.3 AND _alexandria_hull_distance <= 0.025']['https://alexandria.icams.rub.de/pbesol']['data']
len(strucs_alex_end_data)

25672

In [89]:
path_df_agm = Path('df_agm_jan24.pkl')

if path_df_agm.exists():
    with open(path_df_agm, 'rb') as f:
        df_agm = pickle.load(f)
else:
    df_agm = pd.DataFrame.from_dict(
        {doc["id"]: {"structure": optim_Structure(doc).as_pymatgen.as_dict(), 
                     "composition": str(optim_Structure(doc).as_pymatgen.composition),
                     "spg": doc['attributes']['_alexandria_space_group'],
                     "ehull": doc['attributes']['_alexandria_hull_distance'],
                     "bandgap_direct": doc['attributes']['_alexandria_band_gap_direct'],
                     "bandgap": doc['attributes']['_alexandria_band_gap'],
                     "ehull_scan": doc['attributes']['_alexandria_scan_hull_distance'],
                     "bandgap_scan_direct": doc['attributes']['_alexandria_scan_band_gap_direct'],
                     "bandgap_scan": doc['attributes']['_alexandria_scan_band_gap'],
                     } for doc in strucs_alex_end_data}, orient="index"
    )
    with open(path_df_agm, 'wb') as f:
        pickle.dump(df_agm, f)

display(df_agm)

,structure,composition,spg,ehull,bandgap_direct,bandgap,ehull_scan,bandgap_scan_direct,bandgap_scan
agm003211273,"{'@module': 'pymatgen.core.structure', '@class...",Li2 Lu2 F8,88,0.000000,7.9472,7.9367,0.000000,8.7771,8.7771
agm003281172,"{'@module': 'pymatgen.core.structure', '@class...",Lu2 O4 Li2,141,0.000000,4.6777,4.6777,0.000000,5.3714,5.3712
agm002158848,"{'@module': 'pymatgen.core.structure', '@class...",S2 Lu1 Li1,166,0.000000,3.0735,1.8749,0.000000,3.5426,2.3447
agm002158847,"{'@module': 'pymatgen.core.structure', '@class...",S4 Lu2 Li2,141,0.011275,1.8458,1.8458,0.009873,2.3304,2.3304
agm002158986,"{'@module': 'pymatgen.core.structure', '@class...",Lu1 Se2 Li1,166,0.000000,2.8304,1.5140,0.000000,3.2942,1.9660
...,...,...,...,...,...,...,...,...,...
agm003217571,"{'@module': 'pymatgen.core.structure', '@class...",Li4 La4 O8,14,0.001912,3.9811,3.7229,0.012506,4.5299,4.2638
agm003244491,"{'@module': 'pymatgen.core.structure', '@class...",Li2 La2 P8 O24,15,0.000000,5.8361,5.8282,0.000000,6.7026,6.6970
agm003260888,"{'@module': 'pymatgen.core.structure', '@class...",S2 Li2 O2 La2,129,0.000000,3.0738,3.0738,0.000000,3.6983,3.6983
agm003214131,"{'@module': 'pymatgen.core.structure', '@class...",Li2 La2 Ti2 O8,129,0.022372,2.4203,2.3162,0.020680,2.8321,2.7335


In [94]:
agm_structures = []
for agmid, mat in df_agm.iterrows():
    s = Structure.from_dict(mat['structure'])
    agm_structures.append(s)

In [95]:
len(agm_structures)

25672